In [ ]:
from collections import Counter
from typing import Optional

import numpy as np
from flair.data import Sentence
from flair.embeddings import DocumentPoolEmbeddings, WordEmbeddings
from scipy.stats import describe
from tqdm import tqdm

from search_clustering.clustering.knn import *
from search_clustering.embedding import *
from search_clustering.labeling import *
from search_clustering.pipeline import KNNPipeline
from search_clustering.preprocessing import *
from search_clustering.reduction import *
from search_clustering.utils.odp_239 import *
from sklearn.model_selection import ParameterGrid

from pprint import pprint


df = read_odp239_to_df()
data = create_odp239_splits(df)

In [ ]:
from search_clustering.presets.odp_239 import *
from sklearn.metrics import adjusted_rand_score

pipelines = make_pipelines(params_merged)
scores = {}
mean_scores = {}

for identifier, pipeline in tqdm(pipelines.items(), desc="Pipelines"):
    scores_pipe = {}

    for cat in data.keys():
        _, clusters, _ = pipeline.fit_transform(
            data[cat]["data"], verbose=False, visualize=False
        )
        score = adjusted_rand_score(clusters, data[cat]["target"])
        scores_pipe[cat] = score

    scores[identifier] = scores_pipe
    mean_scores[identifier] = np.mean(list(scores_pipe.values()))

pprint(mean_scores)